# Weather Prediction Rater

The following Jupyter Notebook is meant to visualize the differences between Weather Predictions and Weather Measurements

## STEP 1: Load enviornment variables and test API

In [ ]:
import os
from dotenv import load_dotenv

# Load Enviornment variables
load_dotenv() 

In [ ]:

import requests
from urllib.parse import quote

#TODO: Change to desired values
city_name = "New York"

#TODO: API KEY (put key in a .env file)
api_key = os.getenv("weatherApiKey")

# Make initial API call to test if working
url = f"https://api.weatherapi.com/v1/forecast.json?key={quote(api_key)}&q={quote(city_name)}&days=10&aqi=no&alerts=no"
x = requests.get(url)

print(url)
print(x.text)

# STEP 2: Create initial csv file

Below is a code cell that will format a basic csv file to analysis later

Metrics being tracked (each metric will have 240 future hourly prediction points unless modified including the actual reading):
- unix_time
- is_prediction
- temp_f
- is_day
- condition
- wind_mph
- wind_degree
- wind_dir
- pressure_in
- precip_in
- humidity
- cloud
- feelslike_f
- windchill_f
- heatindex_f
- dewpoint_f
- vis_miles
- uv
- gust_mph

In [14]:
# Unix Time
header_vars = ["measurement_time", "utime", "is_prediction", "temp_f", "is_day", "condition", "wind_mph", "wind_degree", "wind_dir", "pressure_in", "precip_in", "humidity", "cloud", "feelslike_f", "windchill_f", "heatindex_f", "dewpoint_f", "vis_miles", "uv", "gust_mph"]

# TODO: Change file name
csv_file_name = "data"

# Set CSV File Header (ONLY RUN ONCE PER FILE)
with open(f"{csv_file_name}.csv", "w") as file:
    header = header_vars[0]
    for var in header_vars[1:]:
        header += "," + var
    header += "\n"
    file.write(header)

## STEP 3: Collect Data

Below are two different cells. The first cell can be left running for however long the user wants and will collect data until the time allocated ends. The second cell will allow the user to manually call the cell however frequently they want to collect their data.

Format of data for both options (csv):

```csv
,utime,is_prediction,temp_f,is_day,condition,...
0,<unix time>,True,80,Partly Cloudly,...
...
```

### OPTION 1: Automatically aggrigate data

In [ ]:
import time
import json
import pandas as pd

# Load Existing Data
df = pd.read_csv(f"{csv_file_name}.csv")

# How long should program run (in hours)
num_time_intervals = 5

# Constant for how long to sleep
secs_per_time_interval = 45 * 60 # 45 mins

# Keep running until num hours
for interval in range(num_time_intervals):
    
    # Make API Request
    response = requests.get(url)
    data = json.loads(response.text)

    """
    Format Current Measurement Data Entry
    """
    cur_measurement = data["current"]
    
    # Change measurment label
    cur_measurement["measurement_time"] = int(data["current"]["last_updated_epoch"]) 
    cur_measurement["utime"] = int(data["current"]["last_updated_epoch"])
    del cur_measurement["last_updated_epoch"]
    del cur_measurement["last_updated"]

    # Only keep condition name
    cur_measurement["condition"] = data["current"]["condition"]["text"]

    # Is Prediction?
    cur_measurement["is_prediction"] = False

    # Check if already added
    if len(df[df["measurement_time"] == cur_measurement["measurement_time"]]) != 0:
        print("NO UPDATE")
        short_time_interval = 20 * 60 # 20 mins
        time.sleep(short_time_interval)
        continue

    # Append to dataframe
    measurement_df = pd.DataFrame([cur_measurement])
    df = pd.concat([df, measurement_df], ignore_index=True)

    """
    Format Prediction Data Entries
    """
    for day in data["forecast"]["forecastday"]:
        for hour_prediction in day["hour"]:

            # Ignore preditions before most recent measurement
            if cur_measurement["measurement_time"] > hour_prediction["time_epoch"]:
                continue

            # Change measurement label
            hour_prediction["measurement_time"] = cur_measurement["measurement_time"] # Time of last valid measurement
            hour_prediction["utime"] = hour_prediction["time_epoch"]                  # Time of predicted
            del hour_prediction["time_epoch"]
            del hour_prediction["time"]


            # Only keep condition name
            hour_prediction["condition"] = hour_prediction["condition"]["text"]

            # Is Prediction?
            hour_prediction["is_prediction"] = True

            # Remove some predictions not in measurement
            del hour_prediction["will_it_rain"]
            del hour_prediction["chance_of_rain"]
            del hour_prediction["will_it_snow"]
            del hour_prediction["chance_of_snow"]
            del hour_prediction["short_rad"]
            del hour_prediction["diff_rad"]

            # Append to DataFrame
            prediction_df = pd.DataFrame([hour_prediction])
            df = pd.concat([df, prediction_df], ignore_index=True)

    # Write to CSV
    df.to_csv(f"{csv_file_name}.csv", index=False)

    # Sleep for an hour
    print(f"Time interval: {interval}")
    time.sleep(secs_per_time_interval)


## OPTION 2: Manually Aggrigate Data

In [ ]:
import json
import sys
import pandas as pd


# Load Existing Data
df = pd.read_csv(f"{csv_file_name}.csv")


# Make API Request
response = requests.get(url)
data = json.loads(response.text)

"""
Format Current Measurement Data Entry
"""
cur_measurement = data["current"]

# Change measurment label
cur_measurement["measurement_time"] = int(data["current"]["last_updated_epoch"]) 
cur_measurement["utime"] = int(data["current"]["last_updated_epoch"])
del cur_measurement["last_updated_epoch"]
del cur_measurement["last_updated"]

# Only keep condition name
cur_measurement["condition"] = data["current"]["condition"]["text"]

# Is Prediction?
cur_measurement["is_prediction"] = False

# Check if already added
if len(df[df["measurement_time"] == cur_measurement["measurement_time"]]) != 0:
    print("NO UPDATE")
    sys.exit(0)

# Append to dataframe
measurement_df = pd.DataFrame([cur_measurement])
df = pd.concat([df, measurement_df], ignore_index=True)

"""
Format Prediction Data Entries
"""
for day in data["forecast"]["forecastday"]:
    for hour_prediction in day["hour"]:

        # Ignore preditions before most recent measurement
        if cur_measurement["measurement_time"] > hour_prediction["time_epoch"]:
                continue

        # Change measurement label
        hour_prediction["measurement_time"] = cur_measurement["measurement_time"] # Time of last valid measurement
        hour_prediction["utime"] = hour_prediction["time_epoch"]                  # Time of predicted
        del hour_prediction["time_epoch"]
        del hour_prediction["time"]


        # Only keep condition name
        hour_prediction["condition"] = hour_prediction["condition"]["text"]

        # Is Prediction?
        hour_prediction["is_prediction"] = True

        # Remove some predictions not in measurement
        del hour_prediction["will_it_rain"]
        del hour_prediction["chance_of_rain"]
        del hour_prediction["will_it_snow"]
        del hour_prediction["chance_of_snow"]
        del hour_prediction["short_rad"]
        del hour_prediction["diff_rad"]

        # Append to DataFrame
        prediction_df = pd.DataFrame([hour_prediction])
        df = pd.concat([df, prediction_df], ignore_index=True)

# Write to CSV
df.to_csv(f"{csv_file_name}.csv", index=False)
print("Success")

# STEP 4: Visualize Data

After you have collected data, run the following code cell to create a visual representation comparing the predictions to the measurements

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Import data from our csv
df = pd.read_csv(f"{csv_file_name}.csv")

# Extract all measurments from df
measurements_df = df[df['is_prediction'] == False]

# Seperate Time and Other units
measurement_time = measurement_df['utime'].tolist()
measurement_temp = measurement_df['temp_f'].tolist()

# Seperate each prediction by measurment time
prediction_dfs = []
for time in measurement_time:
    prediction_dfs.append(df[(df['is_prediction'] == True) & (df['measurement_time'] == time)])
print(len(prediction_dfs))

In [ ]:
# Create matplotlib figure
fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(111)

# Format Unix time on graph to be human-readable
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.AutoDateLocator(maxticks=6))

# Create a connected plot of measured temperatures
human_readable_measurement_time = [datetime.fromtimestamp(ts) for ts in measurement_time]
ax.scatter(human_readable_measurement_time, measurement_temp, c='orange', marker='o', label='measurement')

# Create a connected plot of every all predicted temperature from every 10th measurment time
for i, prediction in enumerate(prediction_dfs):
    predict_time = prediction['utime'].tolist()
    predict_temp = prediction['temp_f'].tolist()
    human_readable_prediction_time = [datetime.fromtimestamp(ts) for ts in predict_time]
    ax.plot(human_readable_prediction_time, predict_temp, c='gray', marker='o', label=f'prediction_{i}')

# Set Axes labels and create legend
plt.xlabel("Date")
plt.ylabel("Temperature (F)")
plt.legend(loc='upper right')

# Show Graph
plt.show()